# 3.1 推論ベースの手法とニューラルネットワーク

In [5]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

単語IDが0の単語を全結合層で変換する例

In [3]:
c = np.array([[1, 0, 0, 0, 0, 0, 0]])
print(c.shape)  # ミニバッチ処理を考えて2次元にしている
W = np.random.randn(7, 3)
h = np.dot(c, W)
print(h)

(1, 7)
[[0.44836394 0.62368717 1.5514608 ]]


MatMulレイヤを使う

In [6]:
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[0.44836394 0.62368717 1.5514608 ]]


# 3.2 シンプルなword2vec (CBOWモデル)

## CBOWモデルの推論処理
**入力層 - 中間層 - 出力層** という単純なモデルを考える
- 入力層と出力層のニューロンの数は単語ベクトルの要素数
- 入力層の数はコンテキストとして与える単語数
- 全ての入力層から中間層への変換は重み$\mathrm{W_{in}}$の全結合層によって，中間層から出力層への変換は重み$\mathrm{W_{out}}$の全結合層によって行われる
- 中間層は各入力層の全結合による変換後の値を平均したもの
- 出力層の値は各単語のスコア．Softmax関数を適用すると各単語の確率になる．
- 学習後の重み$\mathrm{W_{in}}$が単語の分散表現になる．中間層のニューロンの数を入力層よりも少なくすることで，密なベクトルが得られる．

In [9]:
# サンプルデータ
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

# 重み初期化
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# レイヤ生成
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# forward
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

# スコア出力
print(s)

[[-1.29826124  1.5057834  -1.3688423   1.53703969  0.40879317 -0.43425067
   0.56123429]]


## CBOWモデルの学習
ネットワークから出力されるスコアにSoftmax関数を適用することで確率を得ることができる．上の例では，単語IDが0と2の単語と共起する単語を予測するようなタスクを考えている．正解が単語IDが1の単語であれば，s[1]が最も高くなるように重みを調整することになる．